In [1]:
%load_ext lab_black
%cd ../../..

/home/shim/cev/dl/log-analytics


In [24]:
import torch
import numpy as np
import pandas as np
import pickle
from tqdm import tqdm
from collections import defaultdict
import sys

In [6]:
deck1 = torch.load("results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr2e-05-dsver5-os40_2-deck1.pth")
deck2 = torch.load("results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr2e-05-dsver5-os40_2-deck2.pth")

In [7]:
deck1["feat"].shape, deck2["feat"].shape

(torch.Size([472972, 768]), torch.Size([1418916, 768]))

In [41]:
deck1.keys()

dict_keys(['fcfeat', 'tlevel', 'fclevel', 'feat', 'otext'])

In [54]:
db1 = {}
with tqdm(total=deck1["feat"].size(0), ncols=100, file=sys.stdout) as t:
    for i in range(deck1["feat"].size(0)):
        text = deck1["otext"][i]
        if text not in db1:
            db1[text] = {"indices": defaultdict(list)}

        db1[text]["indices"][deck1["tlevel"][i].item()].append(i)
        t.update()

100%|███████████████████████████████████████████████████| 472972/472972 [00:01<00:00, 237462.45it/s]


In [61]:
for text in db1.keys():
    db1[text]["best_level"] = max(db1[text]["indices"], key=lambda x: len(db1[text]["indices"]))

In [62]:
db1[text]

{'indices': defaultdict(list, {0: [472966]}), 'best_level': 0}

In [44]:
text

'month day time localhost kibana: {"type":"error","@timestamp":" date T time Z","tags":["warning","stats-collection"],"pid":821,"level":"error","error":{"message":"No Living connections","name":"Error","stack":"Error: No Living connections at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15) at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7) at process._tickCallback (internal/process/next_tick.js:61:11)"},"message":"No Living connections"}'

In [63]:
deck11 = {k: [] for k in deck1.keys()}
for text in db1.keys():
    level = db1[text]["best_level"]
    idx = db1[text]["indices"][level][0]
    for key in deck1.keys():
        deck11[key].append(deck1[key][idx])

In [67]:
deck11["feat"] = torch.stack(deck11["feat"])
deck11["fcfeat"] = torch.stack(deck11["fcfeat"])
deck11["tlevel"] = torch.stack(deck11["tlevel"])
deck11["fclevel"] = torch.stack(deck11["fclevel"])

In [72]:
torch.save(deck11, "results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr2e-05-dsver5-os40_2-deck11.pth")